We'll write a Decision Tree Classifier, in pure Python. Below each of the methods, I've written a little demo to help explain what it does.

In [17]:
# For Python 2 / 3 compatability
from __future__ import print_function
import pandas as pd

In [18]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.

data = pd.read_csv("titanic.csv")

training_data = []

def checkAdult(age):
    if age>=18:
        return "Adult"
    else:
        return "Child"
    
data["Adult/Child"]=data["Age"].apply(checkAdult)
data.head() 

trainingData=data[["Pclass","Adult/Child","Gender","Survived"]]
trainingData.head()

training_data = trainingData.values.tolist()
# print(training_data)

# training_data = [
#     ['Green', 3, 'Apple'],
#     ['Yellow', 3, 'Apple'],
#     ['Red', 1, 'Grape'],
#     ['Red', 1, 'Grape'],
#     ['Yellow', 3, 'Lemon'],
# ]

[[3, 'Adult', 'male', 0], [1, 'Adult', 'female', 1], [3, 'Adult', 'female', 1], [1, 'Adult', 'female', 1], [3, 'Adult', 'male', 0], [3, 'Child', 'male', 0], [1, 'Adult', 'male', 0], [3, 'Child', 'male', 0], [3, 'Adult', 'female', 1], [2, 'Child', 'female', 1], [3, 'Child', 'female', 1], [1, 'Adult', 'female', 1], [3, 'Adult', 'male', 0], [3, 'Adult', 'male', 0], [3, 'Child', 'female', 0], [2, 'Adult', 'female', 1], [3, 'Child', 'male', 0], [2, 'Child', 'male', 1], [3, 'Adult', 'female', 0], [3, 'Child', 'female', 1], [2, 'Adult', 'male', 0], [2, 'Adult', 'male', 1], [3, 'Child', 'female', 1], [1, 'Adult', 'male', 1], [3, 'Child', 'female', 0], [3, 'Adult', 'female', 1], [3, 'Child', 'male', 0], [1, 'Adult', 'male', 0], [3, 'Child', 'female', 1], [3, 'Child', 'male', 0], [1, 'Adult', 'male', 0], [1, 'Child', 'female', 1], [3, 'Child', 'female', 1], [2, 'Adult', 'male', 0], [1, 'Adult', 'male', 0], [1, 'Adult', 'male', 0], [3, 'Child', 'male', 1], [3, 'Adult', 'male', 0], [3, 'Adult', 'f

In [19]:
# Column labels.
# These are used only to print the tree.
header = ["Pclass", "Adult/Child", "Gender", "Survived"]

In [20]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [22]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [24]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [26]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print 
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

In [30]:
def partition(rows, question):
    """Partitions a dataset.
    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [47]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [39]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column
        for val in values:  # for each value
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [41]:
class Leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [42]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [43]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
# code here
    info, question = find_best_split(rows)
    
    if info == 0 :
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows,question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question,true_branch,false_branch)
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
   

    # If we reach here, we have found a useful feature / value
    # to partition on.

    # Recursively build the true branch.

    # Recursively build the false branch.

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.


In [16]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [48]:
my_tree = build_tree(training_data)

In [56]:
print_tree(my_tree)

Is Gender == male?
--> True:
  Is Pclass >= 2?
  --> True:
    Is Adult/Child == Adult?
    --> True:
      Is Pclass >= 3?
      --> True:
        Predict {0: 182, 1: 28}
      --> False:
        Predict {0: 82, 1: 6}
    --> False:
      Is Pclass >= 3?
      --> True:
        Predict {0: 118, 1: 19}
      --> False:
        Predict {1: 11, 0: 9}
  --> False:
    Is Adult/Child == Adult?
    --> True:
      Predict {0: 61, 1: 36}
    --> False:
      Predict {1: 9, 0: 16}
--> False:
  Is Pclass >= 3?
  --> True:
    Is Adult/Child == Adult?
    --> True:
      Predict {1: 28, 0: 39}
    --> False:
      Predict {1: 44, 0: 33}
  --> False:
    Is Pclass >= 2?
    --> True:
      Is Adult/Child == Adult?
      --> True:
        Predict {1: 56, 0: 6}
      --> False:
        Predict {1: 14}
    --> False:
      Is Adult/Child == Adult?
      --> True:
        Predict {1: 75, 0: 2}
      --> False:
        Predict {1: 16, 0: 1}


In [46]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [50]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [51]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trainingData, test_size = 0.2)

In [57]:
for row in test:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: s. Predicted: {1: '94%', 0: '5%'}
Actual: d. Predicted: {1: '94%', 0: '5%'}
Actual: r. Predicted: {1: '94%', 0: '5%'}
Actual: d. Predicted: {1: '94%', 0: '5%'}
